In [1]:
#transformers

In [3]:
import warnings
warnings.filterwarnings('ignore')

import io
import random
import numpy as np
import mxnet as mx
import gluonnlp as nlp
from bert-tensorflow import data, model

SyntaxError: invalid syntax (<ipython-input-3-9d63f4839ea3>, line 9)

In [6]:

np.random.seed(100)
random.seed(100)
mx.random.seed(10000)
# change `ctx` to `mx.cpu()` if no GPU is available.
ctx = mx.gpu(0)

NameError: name 'mx' is not defined

In [8]:
bert_base, vocabulary = nlp.model.get_model('bert_12_768_12',
                                             dataset_name='book_corpus_wiki_en_uncased',
                                             pretrained=True, ctx=ctx, use_pooler=True,
                                             use_decoder=False, use_classifier=False)
print(bert_base)

NameError: name 'nlp' is not defined

In [9]:
bert_classifier = model.classification.BERTClassifier(bert_base, num_classes=2, dropout=0.1)
# only need to initialize the classifier layer.
bert_classifier.classifier.initialize(init=mx.init.Normal(0.02), ctx=ctx)
bert_classifier.hybridize(static_alloc=True)

# softmax cross entropy loss for classification
loss_function = mx.gluon.loss.SoftmaxCELoss()
loss_function.hybridize(static_alloc=True)

metric = mx.metric.Accuracy()

NameError: name 'model' is not defined

In [10]:

tsv_file = io.open('dev.tsv', encoding='utf-8')
for i in range(5):
    print(tsv_file.readline())

FileNotFoundError: [Errno 2] No such file or directory: 'dev.tsv'

In [11]:
# Skip the first line, which is the schema
num_discard_samples = 1
# Split fields by tabs
field_separator = nlp.data.Splitter('\t')
# Fields to select from the file
field_indices = [3, 4, 0]
data_train_raw = nlp.data.TSVDataset(filename='dev.tsv',
                                 field_separator=field_separator,
                                 num_discard_samples=num_discard_samples,
                                 field_indices=field_indices)
sample_id = 0
# Sentence A
print(data_train_raw[sample_id][0])
# Sentence B
print(data_train_raw[sample_id][1])
# 1 means equivalent, 0 means not equivalent
print(data_train_raw[sample_id][2])

NameError: name 'nlp' is not defined

In [12]:
# Use the vocabulary from pre-trained model for tokenization
bert_tokenizer = nlp.data.BERTTokenizer(vocabulary, lower=True)

# The maximum length of an input sequence
max_len = 128

# The labels for the two classes [(0 = not similar) or  (1 = similar)]
all_labels = ["0", "1"]

# whether to transform the data as sentence pairs.
# for single sentence classification, set pair=False
# for regression task, set class_labels=None
# for inference without label available, set has_label=False
pair = True
transform = data.transform.BERTDatasetTransform(bert_tokenizer, max_len,
                                                class_labels=all_labels,
                                                has_label=True,
                                                pad=True,
                                                pair=pair)
data_train = data_train_raw.transform(transform)

print('vocabulary used for tokenization = \n%s'%vocabulary)
print('%s token id = %s'%(vocabulary.padding_token, vocabulary[vocabulary.padding_token]))
print('%s token id = %s'%(vocabulary.cls_token, vocabulary[vocabulary.cls_token]))
print('%s token id = %s'%(vocabulary.sep_token, vocabulary[vocabulary.sep_token]))
print('token ids = \n%s'%data_train[sample_id][0])
print('valid length = \n%s'%data_train[sample_id][1])
print('segment ids = \n%s'%data_train[sample_id][2])
print('label = \n%s'%data_train[sample_id][3])

NameError: name 'nlp' is not defined

In [13]:
# The hyperparameters
batch_size = 32
lr = 5e-6

# The FixedBucketSampler and the DataLoader for making the mini-batches
train_sampler = nlp.data.FixedBucketSampler(lengths=[int(item[1]) for item in data_train],
                                            batch_size=batch_size,
                                            shuffle=True)
bert_dataloader = mx.gluon.data.DataLoader(data_train, batch_sampler=train_sampler)

trainer = mx.gluon.Trainer(bert_classifier.collect_params(), 'adam',
                           {'learning_rate': lr, 'epsilon': 1e-9})

# Collect all differentiable parameters
# `grad_req == 'null'` indicates no gradients are calculated (e.g. constant parameters)
# The gradients for these params are clipped later
params = [p for p in bert_classifier.collect_params().values() if p.grad_req != 'null']
grad_clip = 1

# Training the model with only three epochs
log_interval = 4
num_epochs = 3
for epoch_id in range(num_epochs):
    metric.reset()
    step_loss = 0
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(bert_dataloader):
        with mx.autograd.record():

            # Load the data to the GPU
            token_ids = token_ids.as_in_context(ctx)
            valid_length = valid_length.as_in_context(ctx)
            segment_ids = segment_ids.as_in_context(ctx)
            label = label.as_in_context(ctx)

            # Forward computation
            out = bert_classifier(token_ids, segment_ids, valid_length.astype('float32'))
            ls = loss_function(out, label).mean()

        # And backwards computation
        ls.backward()

        # Gradient clipping
        trainer.allreduce_grads()
        nlp.utils.clip_grad_global_norm(params, 1)
        trainer.update(1)

        step_loss += ls.asscalar()
        metric.update([label], [out])

        # Printing vital information
        if (batch_id + 1) % (log_interval) == 0:
            print('[Epoch {} Batch {}/{}] loss={:.4f}, lr={:.7f}, acc={:.3f}'
                         .format(epoch_id, batch_id + 1, len(bert_dataloader),
                                 step_loss / log_interval,
                                 trainer.learning_rate, metric.get()[1]))
            step_loss = 0

NameError: name 'nlp' is not defined

In [4]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.utils import np_utils

In [5]:
text = play_string(shakespeare_db[shakespeare_db['Play']=='Hamlet'])
X = []
Y = []
length = len(text)
seq_length = 100
for i in range(0, length-seq_length, 1):
 sequence = text[i:i + seq_length]
 label =text[i + seq_length]
 X.append([char_to_n[char] for char in sequence])
 Y.append(char_to_n[label])

NameError: name 'play_string' is not defined

In [10]:
model = Sequential()
model.add(LSTM(400, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(400))
model.add(Dropout(0.2))
model.add(Dense(Y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

AttributeError: 'list' object has no attribute 'shape'

In [6]:
#These, dear onlooker, are the models we need to import. Their true fate and purpose shall be revealed in time
import re
import operator
import numpy as np
import seaborn as sns
import pandas as pd
import pickle as pickle #data processing
from collections import Counter #tokenization
import keras #ML
#import processing as pr #helper
from sklearn.linear_model import LinearRegression
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score
from keras.preprocessing.text import Tokenizer
import torch
from torch import nn
import torch.nn.functional as F
import gensim, logging
from gensim.models import word2vec
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [7]:
shakespeare_db = pd.read_csv("shakespeare-plays/Shakespeare_data.csv")

In [8]:
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(shakespeare_db['PlayerLine'])

In [9]:
uniques = vectorizer.get_feature_names()

In [10]:
word_array = X.toarray()

In [11]:
def play_string(play):
    string = ""
    for i in play:
        string += i
        string += " "
    return string

In [12]:
def alt_keywords(docs, k):
    lines = [line.rstrip() for line in open('SmartStoplist.txt')]
    stop_list = set(lines)
    vectorizer = TfidfVectorizer(stop_words=stop_list)
    tfidf_matrix = vectorizer.fit_transform(docs)
    tfidf_scores = np.sum(tfidf_matrix,axis=0)
    tfidf_scores = np.ravel(tfidf_scores)
    return sorted(dict(zip(vectorizer.get_feature_names(),tfidf_scores)).items(),key=lambda x:x[1], reverse=True)[:k]

In [13]:
def histogram(words):
    #words_list = re.split("(?:(?:[^a-zA-Z]+')|(?:'[^a-zA-Z]+))|(?:[^a-zA-Z']+)", words)
    words_list = re.split("\W*[^\'\w+\']", words)
    #thanks to Martijn Pieters for the above split
    word_dictionary = {}
    counter = len(words_list)
    i = 0
    while i < counter:
        word_dictionary[words_list[i]] = 1
        n = i+1
        while n < counter:
            if words_list[i] == words_list[n]:
                word_dictionary[words_list[i]] += 1
                words_list.pop(n)
                counter -= 1
                n -= 1
            n += 1
        i += 1
    return word_dictionary
def unique_words(histogram):
    return len(histogram)
def frequency(word, histogram):
    return histogram[word]
def create_sorted_array(histogram):
    new_array = []
    count = 0
    for i in histogram:
        new_array.append([histogram[i]])
        new_array[count].append(i)
        count += 1
    new_array.sort()
    return new_array

In [14]:
def find_alt_key_words(play_name, num):
    play = shakespeare_db[shakespeare_db['Play']==play_name]
    play = play['PlayerLine']
    alt_keys = alt_keywords(play,num)
    return alt_keys

In [15]:
def find_key_words(play_name):
    play = shakespeare_db[shakespeare_db['Play']==play_name]
    play = play['PlayerLine']
    keys = keywords(play,10)
    return keys

In [16]:
def all_the_plays():
    new_play = "Henry IV"
    play_array = []
    for i in shakespeare_db['Play']:
        if i != new_play:
            play_array.append(new_play)
            new_play = i
    return play_array

In [17]:
all_plays = all_the_plays()

In [18]:
play_keywords = [[]] 

In [19]:
play_groups = []

In [20]:
def find_keys():
    for i in all_plays:
        play_words = []
        play_words.append(i)
        play_words.append(find_alt_key_words(i, 50))
        play_keywords.append(play_words)

In [21]:
find_keys()

/usr/local/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:300: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['1971', 'ain', 'aren', 'cornell', 'couldn', 'cs', 'didn', 'doesn', 'don', 'english', 'ftp', 'hadn', 'hasn', 'haven', 'isn', 'list', 'mon', 'pub', 'salton', 'shouldn', 'smart', 'stop', 've', 'wasn', 'weren', 'won', 'word', 'wouldn'] not in stop_words.
  'stop_words.' % sorted(inconsistent))
/usr/local/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:300: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['1971', 'ain', 'aren', 'cornell', 'couldn', 'cs', 'didn', 'doesn', 'don', 'english', 'ftp', 'hadn', 'hasn', 'haven', 'isn', 'list', 'mon', 'pub', 'salton', 'shouldn', 'smart', 'stop', 've', 'wasn', 'weren', 'won', 'word', 'wouldn'] not in stop_words.
  'stop_words.' % sorted(inconsistent))
/usr/local/lib/python3.7

/usr/local/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:300: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['1971', 'ain', 'aren', 'cornell', 'couldn', 'cs', 'didn', 'doesn', 'don', 'english', 'ftp', 'hadn', 'hasn', 'haven', 'isn', 'list', 'mon', 'pub', 'salton', 'shouldn', 'smart', 'stop', 've', 'wasn', 'weren', 'won', 'word', 'wouldn'] not in stop_words.
  'stop_words.' % sorted(inconsistent))
/usr/local/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:300: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['1971', 'ain', 'aren', 'cornell', 'couldn', 'cs', 'didn', 'doesn', 'don', 'english', 'ftp', 'hadn', 'hasn', 'haven', 'isn', 'list', 'mon', 'pub', 'salton', 'shouldn', 'smart', 'stop', 've', 'wasn', 'weren', 'won', 'word', 'wouldn'] not in stop_words.
  'stop_words.' % sorted(inconsistent))
/usr/local/lib/python3.7

In [22]:
def find_groups(keys):
    for i in keys:
        play_thing = [[]]
        for n in i:
            if (play_thing == None) and (i[0] != None):
                play_thing.append([i[0]])
            if n != i[0]:
                for y in n:
                    for x in keys:
                        play_match_words = []
                        for xn in x:
                            if xn != n:
                                if xn != x[0]:
                                    for xy in xn:
                                        if xy[0] == y[0]:
                                            play_match_words.append(xy[0])
                            if (play_match_words != None):
                                play_match = [[x[0]]]
                                play_match[0].append(play_match_words)
                                play_thing[0].append(play_match)
        play_groups.append(play_thing)
                                            

In [23]:
find_groups(play_keywords)

In [24]:
def word_groups(keys):
    word_array = []
    for i in keys:
        for n in i:
            if n != i[0]:
                for y in n:
                    word_in_array = False
                    for w in word_array:
                        if w[0] == y[0]:
                            w.append(i[0])
                            word_in_array = True
                    if word_in_array != True:
                        single_word = [y[0]]
                        single_word.append(i[0])
                        word_array.append(single_word)
    return word_array

In [25]:
w = word_groups(play_keywords)

In [26]:
def play_groups(keys):
    play_array = []
    for i in keys:
        array_within_array = []
        for n in i:
            if n == i[0]:
                array_within_array.append(n)
            else:
                for y in n:
                    for z in w:
                        if y[0] == z[0]:
                            for a in z:
                                repeat = False
                                for b in array_within_array:
                                    if a == b[0]:
                                        repeat = True
                                        b[1] += 1
                                if (repeat != True) and (a != y[0]) and (a != i[0]):
                                    array_within_array.append([a,1])
        play_array.append(array_within_array)
        
    return play_array

In [27]:
p = play_groups(play_keywords)

In [28]:
def grouping_plays(play_group, num):
    group_array = []
    for i in play_group:
        a = []
        for x in i:
            if type(x[1]) == int:
                if x[1] > num:
                    if a == []:
                        a.append(i[0])
                    a.append(x[0])
        if a != []:
            group_array.append(a)
    return group_array

In [29]:
key_words = []

In [30]:
def find_all_keys():
    all_plays = shakespeare_db['PlayerLine']
    keys = keywords(all_plays,5)
    return keys

In [50]:
X_modified = np.reshape(X, (len(X), seq_length, 1))
X_modified = X_modified / float(len(play_string(shakespeare_db[shakespeare_db['Play']=='Hamlet'])))
Y_modified = np_utils.to_categorical(Y)

ValueError: zero-size array to reduction operation maximum which has no identity

In [31]:
def get_text():
    w = shakespeare_db['PlayerLine']
    text = ""
    for i in w:
        text += i
        text += " "
    return text

In [36]:
text = np.reshape(text, (len(text), seq_length, 1))

NameError: name 'seq_length' is not defined

In [ ]:
X_modified = np.reshape(X, (len(X), seq_length, 1))
X_modified = X_modified / float(len(text))
Y_modified = np_utils.to_categorical(Y)

In [35]:
model = Sequential()
model.add(LSTM(400, input_shape=(text.shape[1], text.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(400))
model.add(Dropout(0.2))
model.add(Dense(text.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

AttributeError: 'list' object has no attribute 'shape'